In [98]:
import pandas as pd
import datetime as dt
import sklearn
from sklearn.linear_model import LinearRegression
import numpy as np
from numpy.polynomial.polynomial import polyfit
import matplotlib.pyplot as plt


In [65]:
#convert the date strings to date objects, for plotting
def toDate(series):
    return dt.datetime.strptime(series[:-13], '%Y-%m-%d')
def toDate2(series):
    return dt.datetime.strptime(series[:-9], '%Y-%m-%d')
#shift the year and month back by one to account for discrepencies in billing time
def shift_y(series):

    if series['Month'] == 1:
        return series['Year']-1
    else:
        return series['Year']
def shift_m(series):
    if series['Month'] == 1:
        return 12
    else:
        return series['Month']-1

In [66]:
df = pd.read_csv('~/Downloads/repairs_utilities_all.csv')
df['YearShift'] = df[['Year', 'Month']].apply(shift_y, axis=1)
df['MonthShift'] = df[['Year', 'Month']].apply(shift_m, axis=1)
df['ChargeDate'] = df['ChargeDate'].apply(toDate)

In [67]:
#read in weather
w0 = pd.read_csv("~/Dropbox/CDS-2019-AlbanyHub/ToDatabase/LCD_month.csv")
w = w0.loc[w0['YEAR']>=2012, :].copy()
w['DATE2'] = w['DATE'].apply(toDate2)

In [68]:
#Get consumption and weather normalized by days per month
sr2 = df.loc[:, ['YearShift', 'MonthShift', 'ServiceType', 'address', 'ChargeAmount', 'Consumption']].groupby(by=['YearShift', 'MonthShift', 'ServiceType', 'address'])
sr2 = sr2.sum().reset_index()
days_per_month = [31,28,31,30,31,30,31,31,30,31,30,31]
days_per_month_ly =[31,29,31,30,31,30,31,31,30,31,30,31]
w0['HTDD_norm'] = w0['HTDD']
w0['CLDD_norm'] = w0['CLDD']
sr2['Consumption_norm'] = sr2['Consumption']
#Also print months which have more than one record 
for y in range(2001, 2019):
    for m in range(1,13):
        record_w = w0.loc[(w0['YEAR']==y)&(w0['MONTH']==m), :]
        record_u = sr2.loc[(sr2['YearShift']==y)&(sr2['MonthShift']==m), :]
        if y%4 == 0:
            w0.loc[record_w.index, 'HTDD_norm'] = w0.loc[record_w.index, 'HTDD']/days_per_month_ly[m-1]
            w0.loc[record_w.index, 'CLDD_norm'] = w0.loc[record_w.index, 'CLDD']/days_per_month_ly[m-1]
            sr2.loc[record_u.index, 'Consumption_norm'] = sr2.loc[record_u.index, 'Consumption']/days_per_month_ly[m-1]
        else:
            w0.loc[record_w.index, 'HTDD_norm'] = w0.loc[record_w.index, 'HTDD']/days_per_month[m-1]
            w0.loc[record_w.index, 'CLDD_norm'] = w0.loc[record_w.index, 'CLDD']/days_per_month[m-1]
            sr2.loc[record_u.index, 'Consumption_norm'] = sr2.loc[record_u.index, 'Consumption']/days_per_month[m-1]

In [93]:
new_df = pd.merge(w0, sr2,  how='left', left_on=['YEAR', 'MONTH'], right_on=['YearShift','MonthShift'])
new_df_sub = new_df.loc[(new_df['YEAR']==2012) & (new_df['ServiceType']=='RGAS') & (new_df['address']=='912 JEFFERIES AVE'), :]

In [99]:
x = np.array(new_df_sub['HTDD_norm']).reshape(-1, 1)
y = np.array(new_df_sub['Consumption_norm'])
model = LinearRegression().fit(x, y)
model.score(x, y)

0.9419724860868742

In [ ]:
plt.plot(x=new_df_sub['HTDD_norm'], y=new_df_sub['Consumption_norm'], kind='scatter')
b, m = polyfit(x, y, 1)
plt.plot(x=)

In [88]:
new_df.dropna().to_csv('~/Downloads/project_utilities_dateshift.csv', index=False)